In [ ]:
# Install necessary libraries
!pip install transformers unsloth datasets

In [ ]:
# Mount Google Drive for file access (if needed)
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
from datasets import Dataset, load_from_disk, load_dataset, DatasetDict
from sklearn.preprocessing import LabelEncoder
import numpy as np
import pandas as pd
from transformers import AutoTokenizer

model_ckpt = "distilbert-base-uncased"
tokenizer_name = "distilbert_model"

In [ ]:
input_dataset_path = "/content/drive/MyDrive/arpit_project/qwen_model/dataset_filtered_unsloth/Qwen2-1.5b-bnb-4bit_1024/"
output_dataset_name = "/content/drive/MyDrive/bert/dataset_tokenized_distilbert/"

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
ds = DatasetDict()
for split_name in ['train', 'valid', 'test']:
    split_path = f"{input_dataset_path}/{split_name}"
    ds[split_name] = load_from_disk(split_path)
ds

DatasetDict({
    train: Dataset({
        features: ['oldf', 'patch', 'msg', 'id', 'y', 'idx', 'proj', 'lang', 'text', 'seq_len'],
        num_rows: 36750
    })
    valid: Dataset({
        features: ['oldf', 'patch', 'msg', 'id', 'y', 'idx', 'proj', 'lang', 'text', 'seq_len'],
        num_rows: 5956
    })
    test: Dataset({
        features: ['oldf', 'patch', 'msg', 'id', 'y', 'idx', 'proj', 'lang', 'text', 'seq_len'],
        num_rows: 6023
    })
})

In [ ]:
def create_classification_input(row):

    code_changes = f"""
    Existing code:\n\n

    ```\n
    {row['oldf']}
    ```

    Proposed change:\n\n

    ```\n
    {row['patch']}
    ```
    """

    return {"code": code_changes, "y": row['y']}

In [ ]:
for split_name in ['valid', 'test', 'train']:
    processed_ds = ds[split_name].map(create_classification_input, remove_columns=['oldf', 'patch', 'msg', 'proj', 'lang'])
    output_path = f"{output_dataset_name}/{split_name}"
    print(f"Saving {split_name}")
    processed_ds.save_to_disk(output_path)

Map:   0%|          | 0/5956 [00:00<?, ? examples/s]

Saving valid


Saving the dataset (0/1 shards):   0%|          | 0/5956 [00:00<?, ? examples/s]

Map:   0%|          | 0/6023 [00:00<?, ? examples/s]

Saving test


Saving the dataset (0/1 shards):   0%|          | 0/6023 [00:00<?, ? examples/s]

Map:   0%|          | 0/36750 [00:00<?, ? examples/s]

Saving train


Saving the dataset (0/1 shards):   0%|          | 0/36750 [00:00<?, ? examples/s]

In [ ]:
processed_ds

Dataset({
    features: ['id', 'y', 'idx', 'text', 'seq_len', 'code'],
    num_rows: 36750
})